# Datos

**Maestría de Ciencias de Datos e Información**

**Integrantes:**

    Karen Itzel Velasco Chincoya,
    Martha Karina Terán Botello,
    Yuridiana de Jesús Reyes Delgado,

**Modelos Avanzados en Ciencia de Datos**

**Unidad 1 Actividad Proyecto Integrador**

Diciembre 3, 2022

# Introducción

El uso de las tarjetas de crédito en la actualidad se ha vuelto más común ya que evitan manejar efectivo y permiten realizar compras que se pueden pagar posteriormente, lo que incentiva en mayor medida el consumo. El inconveniente que presentan es que se puede incurrir en incumplimientos de pago, lo que ocasiona pérdidas para los bancos e instituciones que otorgan créditos, por lo que resulta muy útil poder predecir la probabilidad de que un cliente no pague el monto del saldo de su tarjeta de crédito.

La predicción de incumplimiento crediticio en función del análisis del perfil de un cliente permite optimizar las decisiones de préstamo, lo que conduce a una mejor experiencia del cliente y una mejor toma de decisiones por las instituciones que otorgan créditos.

El presente trabajo plantea un modelo para ayudar a gestionar el riesgo de incumplimiento a través de una serie de variables identificadas de cada cliente en cada estado de cuenta para perfilarlo. Dichas variables se clasifican en las siguientes categorías generales:

- Variables de delincuencia
- Variables de gasto
- Variables de pago
- Variables de saldo
- Variables de riesgo






# Desarrollo

En este trabajo se realizó una extracción de una parte de los registros y posteriormente se realiza una limpieza de los mismos para ser utilizados.



In [ ]:
#Se importan las librerías a utilizar

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime

from imblearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn import preprocessing, decomposition, impute
from sklearn.experimental import enable_iterative_imputer


In [ ]:
# Se importan los archivos que serán utilizados desde la plataforma de kaggle
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

## Información 

En este trabajo se uso una muestra de 10,000 registros para los siguientes algoritmos, ya que por el volumen de datos se dificulta su procesamiento. 

**Columnas:** Se eliminaron las columnas que no tuvieran un 75% de valores, eliminando 23 columnas.

**PCA:** Para trabajar con el número de columnas del dataset, se aplica Principal Component Analysis para realizar una reducción.



In [ ]:
# Se cargan los archivos de datos proporcionados con la información crediticia de los usuarios
train_df_sample = pd.read_csv('/content/train_df_sample.csv')
train_label_df = pd.read_csv('/content/train_label_df.csv')
train_df = pd.merge(train_df_sample, train_label_df, how="inner", on=["customer_ID"])
test_df = pd.read_csv('/content/test_df.csv')
sample_submission_df = pd.read_csv('/content/sample_submission_df.csv')

FileNotFoundError: ignored

## Explora los datos del problema


In [ ]:
#Se visualiza información de cada conjunto de datos
print("-------- Archivo de Entrenamiento ----------")
print("Filas:",train_df.shape[0]," Columnas:",train_df.shape[1])
train_df.info()
display(train_df.head())
print("\n")
print("-------- Archivo de Pruebas ----------")
print("Filas:",test_df.shape[0]," Columnas:",test_df.shape[1])
test_df.info()
display(test_df.head())

In [ ]:
#Aquí se agrupan y cuentan los registros por tipo de cliente
clientes = train_df.groupby('target').count().reset_index()
clientes['Conteo'] = clientes['customer_ID']
clientes = clientes.filter(['target','Conteo'])
clientes

In [ ]:
#Se grafica el conteo de tipo de cliente identificado 
plt.figure(figsize=(20,10))
plt.pie(clientes['Conteo'],
        labels=clientes['target'],
        colors=['#19668A','#17C4EA'],
        shadow=True,
        explode=[0, 0.1],
        autopct='%1.1f%%',
        textprops=dict(color="w", weight="bold", size=14)
       )
plt.show()


In [ ]:
#Se identifican los registros por año
anios = pd.to_datetime(test_df['S_2']).dt.year
anios

In [ ]:
#Se genera la gráfica correspondiente para indicar los los diferentes años de información con los que se cuenta.
grafica_barras = sns.displot(anios)

### Describe cual es la relación de tu variable objetivo con las demás variables presentes. 

La relación es que las diferentes variables indicadas de los rubros de **Delincuencia (D), Gasto (S), Pago (P), Saldo (B) y Riesgo (R)** nos permiten calificar la capacidad de pago de un cliente o no.

Los registros que tienen un target=1 (Pago) representa el 25.4% mientras que los que tienen un target=0 (No pago) son el 74.6% 

### Determina cuales son las variables que tienen mayor correlación con tu objetivo. 
 

In [ ]:
#Se define la variable con las columnas con más relación con el objetivo.
columnas_categoricas = train_df[['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64','D_68']]
columnas_categoricas

In [ ]:
columnas_categoricas.isnull().sum()
columnas_categoricas

In [ ]:
#Codificamos etiquetas con valor entre 0 y n-1 y nos devuelve las etiquetas codificadas
LabelEncoder = preprocessing.LabelEncoder()
columnas_categoricas = columnas_categoricas.apply(LabelEncoder.fit_transform)
columnas_categoricas

In [ ]:
#Codificamos las variables categóricas como una única matriz numérica y guardamos en un arreglo
Encoder = preprocessing.OneHotEncoder()
Encoder.fit(columnas_categoricas)
ohl = Encoder.transform(columnas_categoricas).toarray()
ohl

In [ ]:
#Obtenemos el listado de los nombres de las variables categóricas
nombres_columnas_categoricas = Encoder.get_feature_names_out(list(columnas_categoricas.columns))
nombres_columnas_categoricas

In [ ]:
#Generamos el conjunto de datos de las variables categóricas
df_cc = pd.DataFrame(ohl,columns=nombres_columnas_categoricas)
df_cc

In [ ]:
# Se crea el conjunto de datos de entrenamiento con las variables categóricas 
train_df = train_df.drop(columns = columnas_categoricas.columns)
train_df = train_df.join(df_cc)
train_df

In [ ]:
# Se crean los conjuntos de datos de entrenamiento para "x" y para "y"
train_df_x = train_df.drop(columns=['target'])
train_df_y = train_df[['target']]

In [ ]:
#Se crea el conjunto de datos de entrenamiento para aplicar Principal Component Analysis
train_df_PCA = train_df_x.drop(columns=['customer_ID','S_2'])

In [ ]:
# Se genera la variable para la estimación de cada característica a partir de las demás.
mv = impute.IterativeImputer()

In [ ]:
#
train_df_x_impute = mv.fit_transform(train_df_PCA)
train_df_x_impute

In [ ]:
#
train_df_x_impute_df = pd.DataFrame(train_df_x_impute,columns=list(train_df_PCA.columns))

In [ ]:
#Se genera la tabla de correlaciones de las variables categóricas
correlation = columnas_categoricas.corr()
correlation

In [ ]:
#Se grafica la tabla de correlaciones
sns.set(rc = {'figure.figsize':(15,15)})
sns.heatmap(correlation, cmap="YlGnBu", annot=True)
plt.show()

## Aplica ingeniería de características para diseñar nuevas características en tu conjunto de datos. 

Principal Components Analysis consiste en la reducción de las dimensiones permitiendo simplificar la complejidad de los conjuntos de datos con múltiples dimensiones conservando la información. 


In [ ]:
#Aplicamos PCA con 20 componentes 
pca = decomposition.PCA(n_components=20)
train_df_x_impute_pca = pca.fit_transform(train_df_x_impute_df)

In [ ]:
train_df_x_impute_pca = pd.DataFrame(train_df_x_impute_pca,columns=[f'col_{n}' for n in range(train_df_x_impute_pca.shape[1])])
train_df_x_impute_pca

In [ ]:
#Unimos el conjunto de datos de entrenamiento con el modelo PCA resultante.
train_df_pca = train_df_y.join(train_df_x[['customer_ID', 'S_2']]).join(train_df_x_impute_pca)
train_df_pca

## Modela tus datos de dos o más formas diferentes para comparar los resultados. 

En este caso se utilizan los algoritmos de aprendizaje supervisado:

**Random Forest** para la clasificación y regresión. Se generan árboles de decisión sobre los datos seleccionados y se realizan las predicciones sobre cada rama, generando así la mejor solución.

**Regresión logística** para predecir el resultado de una variable categórica en función de las variables independientes.





In [ ]:
#Se generan las variables de entrenamiento "x" y "y"
x = train_df_pca.drop(columns=['target','customer_ID','S_2'])
y = train_df_pca['target']
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.25, stratify=y, random_state=42)

In [ ]:
#Se imprimen los resultados de cada conjunto de entrenamiento y prueba
print("X_train  Filas:",X_train.shape[0]," Columnas:",X_train.shape[1])
print("X_test Filas:",X_test.shape[0]," Columnas:",X_test.shape[1])
print("y_train Filas:",y_train.shape)
print("y_test Filas:",y_test.shape)

In [ ]:
#Se aplica el modelo de clasificación y predicción de Random Forrest
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators = 100) 
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
from sklearn import metrics 
print()
 
# Se calcula la precisión del modelo
print("Accuracy del modelo: ", metrics.accuracy_score(y_test, y_pred))

In [ ]:
#Se aplica el modelo de regresión logística
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 0)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
# Se calcula la precisión del modelo
print("Accuracy del modelo: ", metrics.accuracy_score(y_test, y_pred))

# Conclusiones

Este trabajo permitió poner en práctica lo revisado en los cursos de la plataforma Kaggle: exploración de datos, ingeniería de características y Modelos de ML principalmente, así como la visualización de los datos.

Escogimos un tema donde el conjunto de datos era de un tamaño muy grande que resultaba complejo de manejar al ejecutar este notebook, por lo que se hizo un subconjunto para efectos de la codificación.



# Referencias

*   https://www.kaggle.com/code/girishkumarsahu/american-express-default-prediction-eda/notebook
*   https://www.kaggle.com/code/girishkumarsahu/american-express-default-prediction-ml-model
*   Jamieson Bolder, David. **Credit-Risk Modelling: Theoretical Foundations, Diagnostic Tools, Practical Examples, and Numerical Recipes in Python**. Springer 2018. 